In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np


In [ ]:
events_df = pd.read_csv('./data/retailrocket/events.csv')
category_tree_df = pd.read_csv('./data/retailrocket/category_tree.csv')
item_properties_1_df = pd.read_csv('./data/retailrocket/item_properties_part1.csv')
item_properties_2_df = pd.read_csv('./data/retailrocket/item_properties_part2.csv')

In [ ]:
item_properties_df = pd.concat([item_properties_1_df, item_properties_2_df])


In [ ]:
item_properties_df.itemid.unique().tolist()

In [ ]:
tf.raw_ops.OneHot(indices=item_properties_df.itemid.unique().tolist(),depth=len(item_properties_df.itemid.unique()),on_value=1,off_value=0)

In [ ]:
events_df = events_df.drop(["event","transactionid"],axis=1)
events_df

In [ ]:
events_df.sort_values("visitorid")

In [ ]:
users = events_df['visitorid']
users = users.value_counts()
users

In [ ]:
user_1150086 = events_df.loc[events_df.visitorid==1150086]
user_1150086

In [ ]:
users = events_df['visitorid']
users = users.value_counts()
users = users.iloc[users.values > 32].index
users = users.sort_values()

### 任务：将数据集改成buffer style，即按timestamp的顺序，整理成visitorid, itemid, nextitemid的形式


将events按照visitorid排序，再按照每人访问任一物品的时间顺序排序

In [ ]:
events_df = events_df.sort_values(['visitorid', 'timestamp'])
events_df

去除其中访问物品数小于10的events

In [ ]:
users = events_df['visitorid']
users = users.value_counts()
users = users.iloc[users.values > 10].index
users = users.sort_values()
users

In [ ]:
users_10_length = len(users)
events10 = events_df.loc[events_df.visitorid==users[0]]
for i in range(users_10_length-1):
    events10 = pd.concat([events10, events_df.loc[events_df.visitorid==users[i+1]]])
events10

将nextitemid添加到每个events中

In [ ]:
events10temp = events10
events10temp = events10temp.reset_index(drop=True)

In [ ]:
group = events10temp.groupby('visitorid').groups
group

In [ ]:
events10temp=events10temp.assign(nextitemid=0)

In [ ]:
events10temp

In [ ]:
events_length=len(events10temp.index)
for i in range(events_length):
    if i+1 in group[events10temp.loc[i].visitorid]:
        events10temp.loc[i].nextitemid=events10temp.loc[i+1].itemid
events10temp

In [ ]:
events10temp = events10temp[events10temp.nextitemid != 0]

In [ ]:
events10temp

In [ ]:
events10temp.to_csv(path_or_buf='./data/retailrocket/SLdata.csv')

把itemid对应的property加入到数据里。

### supervised learning


In [1]:
import pandas as pd
import tensorflow as tf
import keras
import numpy as np
from CFN_impl import CFNCell
from d2l import tensorflow as d2l

#### load data

In [2]:
dataset = pd.read_csv('./data/retailrocket/seqData.csv')
dataset

,user,actions
0,54,"[388096, 283115, 38965, 319680, 283115, 388096..."
1,75,"[257575, 257575, 257575, 257575, 257575, 25757..."
2,155,"[134620, 123027, 134620, 50928, 373637, 151670..."
3,172,"[397068, 55710, 464731, 403998, 18519, 27248, ..."
4,224,"[443407, 168697, 443407, 168697, 443407, 16869..."
...,...,...
19577,1407206,"[384514, 162046, 162046, 166941, 454393, 19838..."
19578,1407281,"[69868, 116624, 264521, 174409, 288340, 260581..."
19579,1407340,"[321676, 66145, 148153, 85858, 274436, 248140,..."
19580,1407374,"[399031, 349846, 454301, 397429, 422992, 13997..."


In [3]:
items1 = pd.read_csv('./data/retailrocket/item_properties_part1.csv')
items2 = pd.read_csv('./data/retailrocket/item_properties_part2.csv')
items = pd.concat([items1, items2])

In [4]:
item_size = items.itemid.nunique()
item_size

417053

In [5]:
embed = tf.keras.layers.Embedding(
    input_dim=item_size, 
    output_dim=32,
    embeddings_initializer='uniform',
    input_length=1
)

In [8]:
item_list = items.itemid.unique()
item_list

array([460429, 206783, 395014, ..., 449778, 352709, 170360])

In [15]:
items_embedded = embed(item_list)

In [11]:
lookup = tf.nn.embedding_lookup

In [6]:
class RNNModel(tf.keras.layers.Layer):
    def __init__(self, rnn_layer, item_size, **kwargs):
        super(RNNModel, self).__init__(**kwargs)
        self.rnn = rnn_layer
        self.dense = tf.keras.layers.Dense(32)
        self.dropout = tf.keras.layers.Dropout(0.1)
        self.item_embed = tf.keras.layers.Embedding(
            input_dim=item_size, output_dim=32
        )
        # self.state_embed = tf.keras.layers.Embedding(
        #     input_dim=1, output_dim=32
        # )


    def call(self, inputs, state):
        X = inputs
        #X = self.item_embed(inputs)
        # rnn返回两个以上的值
        Y, *state = self.rnn(X, state)
        output_ = self.dropout(Y)
        output = self.dense(tf.reshape(output_, (-1, Y.shape[-1])))
        return output, state

    def begin_state(self, *args, **kwargs):
        return self.rnn.cell.get_initial_state(*args, **kwargs)

## 现在的问题是，怎么用特征向量表示每个物品。

两种方法：
1. onehot，怎么解决维数过大问题
2. embedding，怎么用。